In [1]:
import pandas as pd

# complete data-range
date0 = pd.to_datetime('2010-01-04')
date1 = pd.to_datetime('2024-11-30')
future = pd.to_datetime('2030-05-16')

def canonical_columns(df):
    """
    Converts the given DataFrame to a canonical format for Bitcoin price analysis.
    
    Parameters:
    - df (pd.DataFrame): The input DataFrame containing Bitcoin price data.
    
    Returns:
    - pd.DataFrame: A copy of the input DataFrame with the following modifications:
      - 'time' column converted to 'date' in datetime format, with time set to midnight.
      - Only 'date', 'time', and 'usd' (formerly 'open') columns are retained.
    """
    df = df.copy()
    df['date'] = pd.to_datetime(df['time'], unit='s')
    df['date'] = pd.to_datetime(df['date'].dt.date)
    df = df[['date', 'open']]
    # df = df[['date', 'time', 'open']]
    df = df.rename(columns={'open': 'usd'})
    return df
 
def extend_dates(df, start_date, end_date):
    """
    Extends the given DataFrame to include future dates and fills missing values with the last known value.
    
    Parameters:
    - df (pd.DataFrame): The input DataFrame containing Bitcoin price data.
    - start_date (pd.Timestamp): The start date for the extension.
    - end_date (pd.Timestamp): The end date for the extension.
    
    Returns:
    - pd.DataFrame: A copy of the input DataFrame with the following modifications:
      - The 'date' column is extended to include future dates.
      - Missing values in the 'usd' column are filled with the last known value.
    """
    df = df.copy()
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    # df = pd.merge(df, pd.DataFrame({'date': date_range}), on='date', how='outer')
    df = pd.merge(df, pd.DataFrame({'date': date_range}), on='date')
    df.sort_values('date', inplace=True)

    # Use forward fill (ffill) to fill missing values with the last known value
    df['usd'] = df['usd'].ffill()
    return df

#btc = pd.read_csv('data/raw/BTCUSD.csv')
btc = pd.read_csv('data/BTCUSD_2024_11_21.csv')
btc = canonical_columns(btc)
btc = extend_dates(btc, date0, future)

# Set the condition to identify rows with dates larger than today
condition = btc['date'] > date1

# Erase data in the 'DataColumn' for dates larger than today
btc.loc[condition, 'usd'] = None  # Replace with None or any other value you prefer

display(btc)

,date,usd
0,2010-01-04,0.000672
1,2010-01-11,0.000695
2,2010-01-18,0.001231
3,2010-01-25,0.002139
4,2010-02-01,0.002707
...,...,...
765,2024-10-28,66070.279524
766,2024-11-04,67757.951012
767,2024-11-11,71065.414256
768,2024-11-18,78529.779628


In [2]:
import numpy as np

def _unix_to_btc_time(date):
    first = 1.2625632e+09
    delta = 1e7
    return date - first + delta

def _bitcoin_time(date):
    x = pd.to_datetime(date).copy()
    x = x.apply(lambda x: x.timestamp())
    return _unix_to_btc_time(x)

def _evaluate_fit(fit, date):
    # date -> btc-time
    x = pd.to_datetime(date)
    x = x.apply(lambda x: x.timestamp())
    x = _unix_to_btc_time(x)
    x = np.log(x)
    return np.exp(fit[0] * x + fit[1])

def btc_fit(dates, price):
    """
    Performs a logscale linear fit on the Bitcoin price data to predict future prices.

    Parameters:
    - dates (pd.Series): A series of dates for which the Bitcoin prices are known.
    - price (pd.Series): A series of Bitcoin prices corresponding to the dates.

    Returns:
    - pd.Series: A series of predicted Bitcoin prices based on the logscale linear fit.
    """
    t = _bitcoin_time(dates)
    t = np.log(t)
    y = np.log(price)
    btc_fit = np.polyfit(t, y, 1)
    return _evaluate_fit(btc_fit, btc['date'])

def _evaluate_fit_time(fit, price, time):
    # date -> btc-time
    return (np.exp((np.log(price) - fit[1]) / fit[0]) - time) / (24 * 60 * 60)

def btc_fit_time(dates, price):
    """
    This function calculates the time until the Bitcoin price reaches a certain value based on a logscale linear fit.
    
    Parameters:
    - dates (pd.Series): A series of dates for which the Bitcoin prices are known.
    - price (pd.Series): A series of Bitcoin prices corresponding to the dates.
    
    Returns:
    - pd.Series: A series of times until the Bitcoin price reaches the specified value.
    """
    t = _bitcoin_time(dates)
    t = np.log(t)
    y = np.log(price)
    btc_fit = np.polyfit(t, y, 1)
    return _evaluate_fit_time(btc_fit, btc['usd'], np.exp(t))

In [3]:
def log_fits(btc, last_included_date):
    # Create a copy of the original DataFrame
    df = btc.copy()
    # Limit the DataFrame to the specified date range
    df = df[df['date'] < pd.to_datetime(last_included_date)].copy()

    # Compute the main fit for the filtered range
    btc['fit'] = btc_fit(df['date'], df['usd'])
    btc['fit_time'] = btc_fit_time(df['date'], df['usd'])

    # Compute undervalued fit
    btc_undervalued = df[df['usd'] < btc.loc[df.index, 'fit']]  # Mask only for relevant indices
    btc['undervalued'] = btc_fit(btc_undervalued['date'], btc_undervalued['usd'])

    # Compute overvalued fit
    btc_overvalued = df[df['usd'] > btc.loc[df.index, 'fit']]  # Mask only for relevant indices
    btc['overvalued'] = btc_fit(btc_overvalued['date'], btc_overvalued['usd'])

    # Compute bubble fit
    btc_bubble = df[df['usd'] > btc.loc[df.index, 'overvalued']]  # Mask only for relevant indices
    btc['bubble'] = btc_fit(btc_bubble['date'], btc_bubble['usd'])

    # Compute top fit
    btc_top = df[df['usd'] > btc.loc[df.index, 'bubble']]  # Mask only for relevant indices
    btc['top'] = btc_fit(btc_top['date'], btc_top['usd'])

    return btc

today = '2024-11-30'
btc = log_fits(btc, last_included_date='2024-11-30')
display(btc)

,date,usd,fit,fit_time,undervalued,overvalued,bubble,top
0,2010-01-04,0.000672,0.000294,20.976614,0.000110,0.000923,0.002510,0.005522
1,2010-01-11,0.000695,0.000393,14.907205,0.000149,0.001224,0.003293,0.007176
2,2010-01-18,0.001231,0.000518,24.666877,0.000197,0.001598,0.004256,0.009193
3,2010-01-25,0.002139,0.000673,35.805475,0.000258,0.002057,0.005427,0.011623
4,2010-02-01,0.002707,0.000862,37.181573,0.000334,0.002615,0.006836,0.014525
...,...,...,...,...,...,...,...,...
765,2024-10-28,66070.279524,65759.864022,5.235747,44261.092739,108492.977302,145400.553706,172714.513424
766,2024-11-04,67757.951012,66175.141310,26.362510,44549.150746,109155.064266,146253.978720,173693.217215
767,2024-11-11,71065.414256,66592.511139,72.899570,44838.715766,109820.347011,147111.324511,174676.219157
768,2024-11-18,78529.779628,67011.981391,179.773213,45129.793766,110488.836894,147972.604106,175663.532659


In [4]:
# Risk
v0 = np.log(btc['undervalued'])
v1 = np.log(btc['top'])
btc['risk'] = (np.log(btc['usd']) - v0) / (v1 - v0)

v0 = btc['undervalued']
v1 = btc['top']
btc['lin_risk'] = (btc['usd'] - v0) / (v1 - v0)
display(btc[['risk', 'lin_risk']])

,risk,lin_risk
0,0.462246,0.103876
1,0.397960,0.077774
2,0.476526,0.114902
3,0.555249,0.165451
4,0.554774,0.167256
...,...,...
765,0.294236,0.169783
766,0.308187,0.179712
767,0.338658,0.201996
768,0.407597,0.255872


In [5]:
from bitcoin_risk.plotter import bitcoin_risk
fig = bitcoin_risk(btc)
fig.write_html('docs/risk.html')
fig.show()

In [6]:
from bitcoin_risk.plotter import bitcoin_plot
fig = bitcoin_plot(btc)
fig.write_html('docs/btc.html', config={'displaylogo': False})
fig.write_image('docs/btc.png')
fig.show()

In [7]:
import plotly.express as px
fig = px.line(btc, x='date', y=['fit_time'],
            #   title='Bitcoin time risk - days until current price equals fair value',
              labels={'date': 'Date', 'open-price': 'Open Price', 
                      'fit': 'Fit', 
                      'undervalued': 'Undervalued',
                      'overvalued': 'Overvalued',
                      'top': 'Bubble Top'})
fig.update_traces(mode='lines', hovertemplate='%{y}')
fig.update_layout(
            shapes=[
            # Green (low risk)
            dict(
                type="rect",
                xref="paper", yref="y",
                x0=0, x1=1, y0=-750, y1=0,  # Adjust y0 and y1 for the "low risk" range
                fillcolor="rgba(0, 255, 0, 0.2)",  # Green with transparency
                layer="below", line_width=0
            ),
            # Orange (medium risk)
            dict(
                type="rect",
                xref="paper", yref="y",
                x0=0, x1=1, y0=0, y1=750,  # Adjust y0 and y1 for the "medium risk" range
                fillcolor="rgba(255, 165, 0, 0.2)",  # Orange with transparency
                layer="below", line_width=0
            ),
            # Red (high risk)
            dict(
                type="rect",
                xref="paper", yref="y",
                x0=0, x1=1, y0=750, y1=1300,  # Adjust y0 and y1 for the "high risk" range
                fillcolor="rgba(255, 0, 0, 0.2)",  # Red with transparency
                layer="below", line_width=0
            )])
fig.write_html('docs/timerisk.html', config={'displaylogo': False})

In [8]:
from bitcoin_risk.plotter import bitcoin_plot_time_risk
btc['risk'] = btc['fit_time']
fig = bitcoin_plot_time_risk(btc)
fig.write_html('docs/btc_time.html', config={'displaylogo': False})
fig.write_image('docs/btc.png')
fig.show()